Here We are importing all required libraries.

In [82]:
#importing required libraries

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import pandas as pd
import numpy as np
import quandl, math
import datetime

#to plot within notebook

import matplotlib.pyplot as plt
%matplotlib inline

#setting figure size

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 30,15

#for normalizing data

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

In [83]:
#Collect the data and read the file

df = pd.read_csv('data/NSE-TECHM.csv')

#print the head

df.head()

,Date,open,high,low,Close,adjusted_close,volume
0,12-04-2019,779.00,786.35,774.55,785.35,785.35,3161818
1,11-04-2019,790.40,795.75,779.50,782.85,782.85,1683293
2,10-04-2019,786.05,798.95,786.00,790.25,790.25,2497013
3,09-04-2019,790.35,797.70,787.30,789.50,789.50,2128462
4,08-04-2019,779.70,790.00,773.40,787.15,787.15,1368825


In [84]:

df['Date'] = pd.to_datetime(df.Date,format='%d-%m-%Y')
df.index = df['Date']

#creating dataframe with date and the target variable
data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])

for i in range(0,len(data)):
     new_data['Date'][i] = data['Date'][i]
     new_data['Close'][i] = data['Close'][i]

new_data.head()

,Date,Close
0,2014-04-07 00:00:00,448.25
1,2014-04-09 00:00:00,434.438
2,2014-04-10 00:00:00,427.05
3,2014-04-11 00:00:00,434.888
4,2014-04-15 00:00:00,441.05


In [85]:
#setting date as an index
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

new_data.head()

,Close
Date,
2014-04-07,448.25
2014-04-09,434.438
2014-04-10,427.05
2014-04-11,434.888
2014-04-15,441.05


In [86]:
#creating train and test sets
dataset = new_data.values

train = dataset[0:987,:]
valid = dataset[987:,:]

print("Length of training data ->> ",len(train))
print("Length of testing data ->> ",len(valid))


Length of training data ->>  987
Length of testing data ->>  247


In [87]:
#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

print("Normalized dataset ->>>>")
scaled_data

Normalized dataset ->>>>


Z:\Anaconda\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


array([[0.15743982],
       [0.12721663],
       [0.11105033],
       ...,
       [0.90579869],
       [0.88960613],
       [0.89507659]])

In [88]:
x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])

x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
print("x_train")
print(x_train)


x_train
[[[0.15743982]
  [0.12721663]
  [0.11105033]
  ...
  [0.33025164]
  [0.32576586]
  [0.33273961]]

 [[0.12721663]
  [0.11105033]
  [0.12820131]
  ...
  [0.32576586]
  [0.33273961]
  [0.33408096]]

 [[0.11105033]
  [0.12820131]
  [0.1416849 ]
  ...
  [0.33273961]
  [0.33408096]
  [0.34578775]]

 ...

 [[0.34474836]
  [0.36280088]
  [0.38380744]
  ...
  [0.52746171]
  [0.52932166]
  [0.52199125]]

 [[0.36280088]
  [0.38380744]
  [0.38172867]
  ...
  [0.52932166]
  [0.52199125]
  [0.54737418]]

 [[0.38380744]
  [0.38172867]
  [0.36641138]
  ...
  [0.52199125]
  [0.54737418]
  [0.56247265]]]


In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=100))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=2, batch_size=2, verbose=2)

Epoch 1/2
 - 94s - loss: 0.0035
Epoch 2/2


In [ ]:
#predicting 246 values, using past 60 from the train data
inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

In [ ]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
rms

In [ ]:
#for plotting
train = new_data[:1234]
valid = new_data[987:]
valid['Predictions'] = closing_price
plt.figure(figsize = (20,10))
plt.xlabel('time', fontsize=15)
plt.ylabel('price', fontsize=15)
plt.plot(train['Close'], color="aqua")
#plt.plot(valid['Close'])
plt.plot(valid['Predictions'], color="black")
plt.legend(fontsize=15)